Libraries required to Install

In [2]:
%pip install google-cloud-documentai
%pip install google-api-core

  Using cached google_cloud_documentai-2.15.0-py2.py3-none-any.whl (236 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached proto_plus-1.22.2-py3-none-any.whl (47 kB)
  Using cached googleapis_common_protos-1.59.1-py2.py3-none-any.whl (224 kB)
  Using cached grpcio_status-1.54.2-py3-none-any.whl (5.1 kB)
     ---------------------------------------- 4.1/4.1 MB 3.0 MB/s eta 0:00:00
     ------------------------------------ 422.5/422.5 kB 976.9 kB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.51.3
    Uninstalling grpcio-1.51.3:
      Successfully uninstalled grpcio-1.51.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.4 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.2 which is incompatible.
rasa 3.4.5 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.2 which is incompatible.
mysql-connector-python 8.0.32 requires protobuf<=3.20.3,>=3.11.0, but you have protobuf 4.23.2 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


## Installing Libraries

In [3]:

from __future__ import annotations

from collections.abc import Sequence

from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore



## setting the credentials, path of the document

In [15]:

project_id = 'llmapps'
location = 'eu' # Format is 'us' or 'eu'
processor_id = 'c0ec8848a9756a33' # Create processor before running sample
file_path = r"C:\Users\Linuxbean\Desktop\13-06-2023\Prakhar Work\Copy of Cola pdf.pdf"
mime_type = 'application/pdf'

## Functions

In [16]:
# def process_document_form_sample(
#     project_id: str, location: str, processor_id: str, file_path: str, mime_type: str
# ) -> None:
#     # Online processing request to Document AI
#     document = process_document(
#         project_id, location, processor_id, file_path, mime_type
#     )

#     # Read the table and form fields output from the processor
#     # The form processor also contains OCR data. For more information
#     # on how to parse OCR data please see the OCR sample.

#     # For a full list of Document object attributes, please reference this page:
#     # https://cloud.google.com/python/docs/reference/documentai/latest/google.cloud.documentai_v1.types.Document

#     text = document.text
#     print(f"Full document text: {repr(text)}\n")
#     print(f"There are {len(document.pages)} page(s) in this document.")

#     # Read the form fields and tables output from the processor
#     for page in document.pages:
#         print(f"\n\n**** Page {page.page_number} ****")

#         print(f"\nFound {len(page.tables)} table(s):")
#         for table in page.tables:
#             num_collumns = len(table.header_rows[0].cells)
#             num_rows = len(table.body_rows)
#             print(f"Table with {num_collumns} columns and {num_rows} rows:")

#             # Print header rows
#             print("Columns:")
#             print_table_rows(table.header_rows, text)
#             # Print body rows
#             print("Table body data:")
#             print_table_rows(table.body_rows, text)

#         print(f"\nFound {len(page.form_fields)} form field(s):")
#         for field in page.form_fields:
#             name = layout_to_text(field.field_name, text)
#             value = layout_to_text(field.field_value, text)
#             print(f"    * {repr(name.strip())}: {repr(value.strip())}")


def process_document(
    project_id: str, location: str, processor_id: str, file_path: str, mime_type: str
) -> documentai.Document:
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor, e.g.:
    # projects/project_id/locations/location/processor/processor_id
    name = client.processor_path(project_id, location, processor_id)

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load Binary Data into Document AI RawDocument Object
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)

    # Configure the process request
    request = documentai.ProcessRequest(name=name, raw_document=raw_document)

    result = client.process_document(request=request)

    return result.document


def print_table_rows(
    table_rows: Sequence[documentai.Document.Page.Table.TableRow], text: str
) -> None:
    for table_row in table_rows:
        row_text = ""
        for cell in table_row.cells:
            cell_text = layout_to_text(cell.layout, text)
            row_text += f"{repr(cell_text.strip())} | "
        print(row_text)


def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document's text. This function converts
    offsets to a string.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in layout.text_anchor.text_segments:
        start_index = int(segment.start_index)
        end_index = int(segment.end_index)
        response += text[start_index:end_index]
    return response

In [17]:
def process_document_form_sample(
    project_id: str, location: str, processor_id: str, file_path: str, mime_type: str
) -> None:
    # Online processing request to Document AI
    document = process_document(
        project_id, location, processor_id, file_path, mime_type
    )

    # Read the table and form fields output from the processor
    # The form processor also contains OCR data. For more information
    # on how to parse OCR data please see the OCR sample.

    # For a full list of Document object attributes, please reference this page:
    # https://cloud.google.com/python/docs/reference/documentai/latest/google.cloud.documentai_v1.types.Document

    text = document.text
    # print(f"Full document text: {repr(text)}\n")
    # print(f"There are {len(document.pages)} page(s) in this document.")

    for page in document.pages:
        page_number = page.page_number
        page_text = layout_to_text(page.layout, text)

        print(f"\n\n**** Page {page_number} ****")
        print(f"Page text: {repr(page_text)}")

        # Read the form fields and tables output from the processor
        print(f"\nFound {len(page.tables)} table(s):")
        for table in page.tables:
            num_columns = len(table.header_rows[0].cells)
            num_rows = len(table.body_rows)
            print(f"Table with {num_columns} columns and {num_rows} rows:")

            # Print header rows
            print("Columns:")
            print_table_rows(table.header_rows, text)
            # Print body rows
            print("Table body data:")
            print_table_rows(table.body_rows, text)

        print(f"\nFound {len(page.form_fields)} form field(s):")
        for field in page.form_fields:
            name = layout_to_text(field.field_name, text)
            value = layout_to_text(field.field_value, text)
            print(f"    * {repr(name.strip())}: {repr(value.strip())}")


In [18]:
process_document_form_sample(project_id, location, processor_id, file_path, mime_type)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.